In [1]:
import ccxt
import requests
from binance.client import Client
from binance.exceptions import BinanceAPIException
from datetime import datetime, timedelta
import time
import hashlib
import hmac
import pandas
import json


In [36]:
def get_keys(filepath):
    with open(filepath) as config_file:
        config = json.load(config_file)
        api_key = config['api_key']
        api_secret=config['api_secret']
    return api_key,api_secret

In [37]:
get_keys("binance_keys.json")

('dIgzgeXGUjuNi81pWAZAzHt1zkYcfoN4QM0oasQsVkJQmoqdkC7dilNLiETRheyU',
 'ZxiMdpOSzIirGkhnkEBJsNHZ92okUqBwSulFIKKeHSLbLdlPkjWN9lMx5lsJn79g')

In [2]:
def information_for_options(client):
    markets = client.fetch_markets()
    server_time = client.fetch_time()
    # Filter option markets
    option_markets = [market for market in markets if market['type'] == 'option']
    # print("Option markets:", option_markets)
    for i in option_markets:
        # print(i["info"])
        filtered_list = []
        expiry_timestamp = float(i['info']['expiration_timestamp'])/1000
        # Convert expiry timestamp to datetime
        expiry_datetime = datetime.utcfromtimestamp(expiry_timestamp)

        # Calculate current datetime plus 20 weeks
        current_datetime = datetime.utcnow()
        twenty_weeks_later = current_datetime + timedelta(weeks=20)
        # Check if expiry datetime is greater than 20 weeks later
        if expiry_datetime <= twenty_weeks_later:
            filtered_list.append(i)
            # if call option and strike is not none, add to list
            second_filtered_list = []
            for item in filtered_list:
                # if item["strike"] is not None and 
                if item["optionType"] == "call" and item["strike"] is not None:

                    second_filtered_list.append(item)
                    # third_filtered_list = []
                    # # Loop through second filtered list and get contract with longest expiration
                    # for item in second_filtered_list:
                    #     expired_time = item["expiration_timestamp"]
                        
    #                     if item["expiration_timestamp"] is not None:
    #                         third_filtered_list.append(item)
            third_filtered_list = []
            expiry_timestamp_list = []
            if len(second_filtered_list) > 0:
                longest_expiry_contract = max(second_filtered_list, key=lambda x: x['expiry'])

    # print(longest_expiry_contract)
    # Contract symbol
    symbol = longest_expiry_contract["id"]
    # Timestamp of contract            
    expire_time = float(longest_expiry_contract["info"]["expiration_timestamp"])
    # convert expiration timestamp to datetime
    expiry_datetime = datetime.utcfromtimestamp(expire_time/1000)
    # Get delta
    contract_delta = client.fetch_greeks(longest_expiry_contract["symbol"])
    delta = contract_delta["info"]["greeks"]["delta"]

    # Get contract size
    # Calculate share to purchase
    contract_size = longest_expiry_contract["info"]["contract_size"]
    share_to_purchase = float(delta) * float(contract_size)


    return expiry_datetime, share_to_purchase, symbol ,delta ,contract_size

In [3]:

mode = "test"

client_id_testnet = "m_cYdGRs"
client_secret_testnet = "CFMj4XERdBpYU5f-xGiEeM6q29WWPh78KbOllO8I5nI"

client_id_realnet = "VUSrWKNX"
client_secret_realnet = "CNIEmjiKy2p-h28O4Mda1QKD8hXJ3duA5rAODdLfvwE"

if mode == "test":
    client_id = client_id_testnet
    client_secret = client_secret_testnet
    base_url = 'https://test.deribit.com/api/v2/' # test base url
else:
    client_id = client_id_realnet
    client_secret = client_secret_realnet
    base_url = 'https://www.deribit.com/api/v2/'

print(base_url)


# Function to get access token
def get_access_token(client_id, client_secret):
    url = f'{base_url}public/auth'
    params = {
        'client_id': client_id,
        'client_secret': client_secret,
        'grant_type': 'client_credentials'
    }
    response = requests.get(url, params=params)
    response_data = response.json()
    if 'result' in response_data:
        return response_data['result']['access_token']
    else:
        raise Exception('Failed to get access token: ' + response_data.get('error', {}).get('message', 'Unknown error'))

# Function to fetch account information
def fetch_account_info(access_token):
    url = f'{base_url}private/get_account_summary'
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    params = {
        'currency': 'BTC'  # or 'ETH', etc.
    }
    response = requests.get(url, headers=headers, params=params)
    response_data = response.json()
    if 'result' in response_data:
        return response_data['result']
    else:
        raise Exception('Failed to fetch account information: ' + response_data.get('error', {}).get('message', 'Unknown error'))
    
client = ccxt.deribit({
    'apiKey': client_id,
    'secret': client_secret,
    'timeout': 50000,
})
if mode=="test":
    client.set_sandbox_mode(True)

https://test.deribit.com/api/v2/


In [4]:
access_token = get_access_token(client_id, client_secret)
account_info = fetch_account_info(access_token)
print(account_info)

{'futures_session_upl': 0.0, 'session_rpl': 0.0, 'delta_total_map': {}, 'portfolio_margining_enabled': False, 'projected_maintenance_margin': 0.0, 'projected_initial_margin': 0.0, 'cross_collateral_enabled': False, 'estimated_liquidation_ratio': 0.0, 'options_value': 0.0, 'currency': 'BTC', 'available_withdrawal_funds': 100.0, 'options_pl': 0.0, 'maintenance_margin': 0.0, 'options_theta': 0.0, 'available_funds': 100.0, 'balance': 100.0, 'options_session_upl': 0.0, 'options_vega': 0.0, 'margin_balance': 100.0, 'additional_reserve': 0.0, 'initial_margin': 0.0, 'fee_balance': 0.0, 'futures_pl': 0.0, 'futures_session_rpl': 0.0, 'change_margin_model_api_limit': {'rate': 10, 'timeframe': 86400000}, 'options_gamma': 0.0, 'options_gamma_map': {}, 'options_session_rpl': 0.0, 'options_vega_map': {}, 'options_theta_map': {}, 'projected_delta_total': 0.0, 'limits': {'non_matching_engine': {'burst': 15000, 'rate': 20}, 'limits_per_currency': False, 'matching_engine': {'trading': {'total': {'burst':

In [5]:

def create_option_order(client):
    try:
        # Construct the order
        symbol=information_for_options(client)[-3]
        order = client.create_market_buy_order(symbol,1)

        print(f"Order created successfully: {order}")
        return order

    except Exception as e:
        print(f"Failed to create order: {e}")
        return None

# Example usage

In [6]:
expiry_datetime, share_to_purchase, symbol ,delta ,contract_size= information_for_options(client)
print(expiry_datetime,share_to_purchase, symbol)
create_option_order(client)

2024-09-27 08:00:00 0.0009 ETH-27SEP24-17000-C
Order created successfully: {'info': {'is_liquidation': False, 'risk_reducing': False, 'creation_timestamp': '1718546233776', 'order_type': 'limit', 'order_state': 'open', 'original_order_type': 'market', 'contracts': '1.0', 'average_price': '0.0', 'reduce_only': False, 'post_only': False, 'last_update_timestamp': '1718546233776', 'filled_amount': '0.0', 'replaced': False, 'web': False, 'api': True, 'mmp': False, 'instrument_name': 'ETH-27SEP24-17000-C', 'order_id': 'ETH-8879185259', 'max_show': '1.0', 'time_in_force': 'good_til_cancelled', 'direction': 'buy', 'amount': '1.0', 'price': '0.015', 'label': '', 'trades': []}, 'id': 'ETH-8879185259', 'clientOrderId': None, 'timestamp': 1718546233776, 'datetime': '2024-06-16T13:57:13.776Z', 'lastTradeTimestamp': None, 'symbol': 'ETH/USD:ETH-240927-17000-C', 'type': 'limit', 'timeInForce': 'GTC', 'postOnly': False, 'side': 'buy', 'price': 0.015, 'stopPrice': None, 'triggerPrice': None, 'amount': 

{'info': {'is_liquidation': False,
  'risk_reducing': False,
  'creation_timestamp': '1718546233776',
  'order_type': 'limit',
  'order_state': 'open',
  'original_order_type': 'market',
  'contracts': '1.0',
  'average_price': '0.0',
  'reduce_only': False,
  'post_only': False,
  'last_update_timestamp': '1718546233776',
  'filled_amount': '0.0',
  'replaced': False,
  'web': False,
  'api': True,
  'mmp': False,
  'instrument_name': 'ETH-27SEP24-17000-C',
  'order_id': 'ETH-8879185259',
  'max_show': '1.0',
  'time_in_force': 'good_til_cancelled',
  'direction': 'buy',
  'amount': '1.0',
  'price': '0.015',
  'label': '',
  'trades': []},
 'id': 'ETH-8879185259',
 'clientOrderId': None,
 'timestamp': 1718546233776,
 'datetime': '2024-06-16T13:57:13.776Z',
 'lastTradeTimestamp': None,
 'symbol': 'ETH/USD:ETH-240927-17000-C',
 'type': 'limit',
 'timeInForce': 'GTC',
 'postOnly': False,
 'side': 'buy',
 'price': 0.015,
 'stopPrice': None,
 'triggerPrice': None,
 'amount': 1.0,
 'cost':

In [7]:
share_to_purchase,symbol,delta,contract_size

(0.0009, 'ETH-27SEP24-17000-C', '0.0009', '1.0')

In [8]:
symbol=symbol.split("-")[0]
symbol=symbol+"USDT"
symbol

'ETHUSDT'

In [38]:
def create_hedging_order(symbol, side,share_to_purchase):
    """
    Create a hedging order on Binance testnet.

    Parameters:
    api_key (str): Your Binance testnet API key.
    api_secret (str): Your Binance testnet API secret.
    symbol (str): The trading pair symbol (e.g., 'BTCUSDT').
    side (str): 'BUY' or 'SELL' order.
    share_to_purchase (float): Amount of the asset to purchase.

    Returns:
    dict: Response from Binance API with order details.
    """
    # Connect to Binance testnet
    api_key,api_secret=get_keys("binance_keys.json")
    client = Client(api_key, api_secret, testnet=True)
    # Get current price of the symbol
    ticker = client.get_symbol_ticker(symbol=symbol)
    current_price = float(ticker['price'])
    print(f"Current price of {symbol}: {current_price}")
    print(f"Share to purchase: {share_to_purchase} ")
    # Create a market order
    try:
        order = client.create_order(
            symbol=symbol,
            side=side,
            type='MARKET',
            quantity=float (share_to_purchase)
        )
        print("Order placed successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")
        order = None
    
    return order



In [39]:
def get_min_contract_size(symbol):
    url = "https://api.binance.com/api/v3/exchangeInfo"
    response = requests.get(url)
    data = response.json()

    for s in data['symbols']:
        if s['symbol'] == symbol:
            for filter in s['filters']:
                if filter['filterType'] == 'LOT_SIZE':
                    min_qty = filter['minQty']
                    return float(min_qty)
    return None

In [40]:
min_contract_size = get_min_contract_size(symbol)

if min_contract_size:
    print(f"The minimum contract size for {symbol} is {min_contract_size}")
else:
    print(f"Symbol {symbol} not found")

The minimum contract size for ETHUSDT is 0.0001


In [41]:
def calculate_and_place_order(symbol,old_delta):
    old_delta=float(old_delta)
    new_delta=float(information_for_options(client)[-2])
    contract_size=float(information_for_options(client)[-1])
    share_to_purchase = (new_delta - old_delta) * contract_size
    if share_to_purchase != 0:  # Only place an order if there's a change in delta
        if share_to_purchase <get_min_contract_size(symbol)*10:
            print(f"Share to purchase too small: {share_to_purchase}")
        else:
            if new_delta > old_delta:
                side = "BUY"
            else:
                side = "SELL"
            create_hedging_order(symbol,side, abs(share_to_purchase))
    else:
        print(f"{datetime.now()}: No change in delta. No order placed.")


In [42]:
create_hedging_order(symbol,side="SELL",share_to_purchase=get_min_contract_size(symbol)*100)

Current price of ETHUSDT: 3591.6
Share to purchase: 0.01 
Order placed successfully.


{'symbol': 'ETHUSDT',
 'orderId': 4403366,
 'orderListId': -1,
 'clientOrderId': 'lNw7NRgS6q9pDv6DbJgJPl',
 'transactTime': 1718552760190,
 'price': '0.00000000',
 'origQty': '0.01000000',
 'executedQty': '0.01000000',
 'cummulativeQuoteQty': '35.91550000',
 'status': 'FILLED',
 'timeInForce': 'GTC',
 'type': 'MARKET',
 'side': 'SELL',
 'workingTime': 1718552760190,
 'fills': [{'price': '3591.55000000',
   'qty': '0.01000000',
   'commission': '0.00000000',
   'commissionAsset': 'USDT',
   'tradeId': 482807}],
 'selfTradePreventionMode': 'EXPIRE_MAKER'}

In [14]:
def get_price_filters(symbol):
    """Retrieve price filter info for a given symbol."""
    base_url = 'https://testnet.binance.vision'
    endpoint = '/api/v3/exchangeInfo'
    response = requests.get(base_url + endpoint)
    data = response.json()

    for symbol_info in data['symbols']:
        if symbol_info['symbol'] == symbol:
            for filter in symbol_info['filters']:
                if filter['filterType'] == 'PRICE_FILTER':
                    return {
                        'minPrice': float(filter['minPrice']),
                        'maxPrice': float(filter['maxPrice']),
                        'tickSize': float(filter['tickSize'])
                    }
    raise Exception(f'Price filters not found for symbol: {symbol}')

In [48]:
get_price_filters(symbol)

{'minPrice': 0.01, 'maxPrice': 1000000.0, 'tickSize': 0.01}

In [43]:
def create_limit_order(symbol, side,share_to_purchase,limit_delta):
    """
    Create a hedging order on Binance testnet.

    Parameters:
    api_key (str): Your Binance testnet API key.
    api_secret (str): Your Binance testnet API secret.
    symbol (str): The trading pair symbol (e.g., 'BTCUSDT').
    side (str): 'BUY' or 'SELL' order.
    share_to_purchase (float): Amount of the asset to purchase.

    Returns:
    dict: Response from Binance API with order details.
    """
    # Connect to Binance testnet
    api_key,api_secret=get_keys("binance_keys.json")
    timestamp = int(time.time() * 1000)
    client = Client(api_key, api_secret, testnet=True)
    # Get current price of the symbol
    ticker = client.get_symbol_ticker(symbol=symbol)
    current_price = float(ticker['price'])
    print(f"Current price of {symbol}: {current_price}")
    print(f"Share to purchase: {share_to_purchase} ")
    print(f"Limited pirce : {round(current_price-current_price*limit_delta/100)}")
    # Create a market order
    try:
        order = client.create_order(
            symbol=symbol,
            side=side,
            type='LIMIT',
            price=round(current_price-current_price*limit_delta/100),
            timeInForce="GTC",
            timestamp=timestamp,
            quantity=float (share_to_purchase)
        )
        print("Order placed successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")
        order = None
    
    return order

In [45]:


# Function to create a signature for the request
def create_signature(query_string, secret_key):
    return hmac.new(secret_key.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

# Function to get all open orders from Binance Spot Testnet
def get_all_open_orders():
    api_key,api_secret=get_keys("binance_keys.json")
    base_url = 'https://testnet.binance.vision'
    endpoint = '/api/v3/openOrders'
    timestamp = int(time.time() * 1000)
    query_string = f'timestamp={timestamp}'

    # Creating a signature
    signature = create_signature(query_string, api_secret)
    query_string += f'&signature={signature}'

    # Header with API key
    headers = {
        'X-MBX-APIKEY': api_key
    }

    # Sending the GET request
    url = f'{base_url}{endpoint}?{query_string}'
    response = requests.get(url, headers=headers)

    # Handling the response
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f'Error: {response.status_code}, Message: {response.text}')
def get_all_past_orders(symbol, start_time=None, end_time=None, limit=500):
    """Retrieve all past orders for a given symbol on Binance Spot Testnet."""
    base_url = 'https://testnet.binance.vision'
    endpoint = '/api/v3/allOrders'
    timestamp = int(time.time() * 1000)
    api_key,api_secret=get_keys("binance_keys.json")  
    # Create query string
    query_string = f'symbol={symbol}&timestamp={timestamp}&limit={limit}'
    if start_time:
        query_string += f'&startTime={start_time}'
    if end_time:
        query_string += f'&endTime={end_time}'

    # Create signature
    signature = create_signature(query_string, api_secret)
    query_string += f'&signature={signature}'

    # Headers
    headers = {'X-MBX-APIKEY': api_key}

    # Send GET request
    url = f'{base_url}{endpoint}?{query_string}'
    response = requests.get(url, headers=headers)

    # Handle the response
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception(f'Error: {response.status_code}, Message: {response.text}')


In [46]:
past_orders=get_all_past_orders(symbol)
past_orders_df=pandas.DataFrame(past_orders)
past_orders_df.to_csv("past_order.csv")

In [47]:
import schedule
import time

# Assuming these are updated dynamically from a data source

with open("old_delta.json", 'r') as file:

    data = json.load(file)
old_delta=data
def job():
    try: 
        global old_delta
        print("Performing job")
        calculate_and_place_order(symbol,old_delta)
        old_delta=information_for_options(client)[-2]
        print(f"Updated old_delta to: {old_delta}")       
        with open("old_delta.json", "w") as json_file:
            json.dump(old_delta, json_file)
    except Exception as e:
        print(f"Exception: {e}")

    
# Schedule the job every hour
schedule.every(10).seconds.do(job)

# Run the scheduler
while True:
    schedule.run_pending()
    time.sleep(1)

Performing job
2024-06-16 22:46:53.407615: No change in delta. No order placed.
Updated old_delta to: 0.0009
Performing job
2024-06-16 22:46:57.801638: No change in delta. No order placed.
Updated old_delta to: 0.0009
Performing job


KeyboardInterrupt: 

In [27]:
def get_account_balances():
    api_key = 'dIgzgeXGUjuNi81pWAZAzHt1zkYcfoN4QM0oasQsVkJQmoqdkC7dilNLiETRheyU'
    api_secret = 'ZxiMdpOSzIirGkhnkEBJsNHZ92okUqBwSulFIKKeHSLbLdlPkjWN9lMx5lsJn79g'

    client = Client(api_key, api_secret, testnet=True)
    try:
        # Fetch account information
        account_info = client.get_account()

        # Extract balances
        balances = account_info['balances']
        balance_info = []

        # Iterate through each balance and append non-zero balances to the list
        for balance in balances:
            asset = balance['asset']
            free_balance = float(balance['free'])
            locked_balance = float(balance['locked'])
            total_balance = free_balance + locked_balance

            if total_balance > 0:
                balance_info.append({
                    'Asset': asset,
                    'Free': free_balance,
                    'Locked': locked_balance,
                    'Total': total_balance
                })

        return balance_info

    except Exception as e:
        print(f"An error occurred: {e}")
        return []

In [28]:
def display_balances(balances):
    if not balances:
        print("No balances found or an error occurred.")
        return

    print(f"{'Asset':<10} {'Free Balance':<15} {'Locked Balance':<15} {'Total Balance':<15}")
    print("="*55)
    for balance in balances:
        print(f"{balance['Asset']:<10} {balance['Free']:<15} {balance['Locked']:<15} {balance['Total']:<15}")

In [29]:
balances = get_account_balances()
display_balances(balances)

Asset      Free Balance    Locked Balance  Total Balance  
ETH        2.97            0.0             2.97           
BTC        1.02604         0.0             1.02604        
LTC        5.0             0.0             5.0            
BNB        1.0             0.0             1.0            
USDT       1050.3974307    0.0             1050.3974307   
TRX        4386.0          0.0             4386.0         
XRP        948.0           0.0             948.0          
NEO        33.0            0.0             33.0           
QTUM       141.0           0.0             141.0          
EOS        617.0           0.0             617.0          
SNT        14258.0         0.0             14258.0        
BNT        606.0           0.0             606.0          
GAS        98.0            0.0             98.0           
LRC        1850.0          0.0             1850.0         
ZRX        931.0           0.0             931.0          
KNC        749.0           0.0             749.0        